In [6]:
import plotly.express as px
import polars as pl
from itables import show
import altair as alt

data_pus = pl.read_csv("data/data_pus.csv")
data_mix = pl.read_csv("data/data_mix_kontra.csv")

def bulan_hingga(bulan_terpilih):
    daftar_bulan = ["JANUARI", "FEBRUARI", "MARET", "APRIL", "MEI", "JUNI", "JULI", "AGUSTUS", "SEPTEMBER", "OKTOBER", "NOVEMBER", "DESEMBER"]
    index = daftar_bulan.index(bulan_terpilih)
    return daftar_bulan[:index + 1]

filter_kabupaten = ["PASANGKAYU", "PASANGKAYU", "MAMUJU", "MAMUJU"]
filter_kecamatan = ["PASANGKAYU", "PASANGKAYU", "PAPALANG", "PAPALANG"]
filter_desa = ["AKO", "GUNUNG SARI", "BODA-BODA", "SISANGO"]
filter_bulan = 'FEBRUARI'

In [3]:
show(
    data_pus.filter(
        pl.col("KABUPATEN").is_in(filter_kabupaten),
        pl.col("KECAMATAN").is_in(filter_kecamatan),
        pl.col("KELURAHAN").is_in(filter_desa),
        pl.col("BULAN").is_in(bulan_hingga(filter_bulan))
    ).group_by(
        'PROVINSI', 'BULAN'
    ).agg([
        pl.col("PUS").sum()
    ])
)

In [4]:
aggregated =  data_pus.filter(
        pl.col("KABUPATEN").is_in(filter_kabupaten),
        pl.col("KECAMATAN").is_in(filter_kecamatan),
        pl.col("KELURAHAN").is_in(filter_desa),
        pl.col("BULAN").is_in(bulan_hingga(filter_bulan))
    ).group_by(
        'PROVINSI', 'BULAN'
    ).agg([
        pl.col("PUS").sum()
    ])

aggregated

PROVINSI,BULAN,PUS
str,str,i64
"""SULAWESI BARAT""","""FEBRUARI""",1786
"""SULAWESI BARAT""","""JANUARI""",1769


In [14]:
# Urutan bulan yang benar
month_order = {
    "JANUARI": 1,
    "FEBRUARI": 2,
    "MARET": 3,
    "APRIL": 4,
    "MEI": 5,
    "JUNI": 6,
    "JULI": 7,
    "AGUSTUS": 8,
    "SEPTEMBER": 9,
    "OKTOBER": 10,
    "NOVEMBER": 11,
    "DESEMBER": 12
}

# Proses data dengan Polars
df_processed = (
    aggregated
    .with_columns(
        pl.col("BULAN").replace(bulan_order).alias("month_order"),
        # Format angka dengan pemisah titik (.) untuk tooltip
        pl.col("PUS").map_elements(
            lambda x: f"{x:,}".replace(",", "."),
            return_dtype=pl.Utf8
        ).alias("PUS_formatted")
    )
    .sort("month_order")
)

# Hitung batas y-axis
min_value = df_processed["PUS"].min()
max_value = df_processed["PUS"].max()
y_min = min_value - (min_value * 0.05)
y_max = max_value + (max_value * 0.05)

# Konversi ke Pandas untuk Altair
df_for_viz = df_processed.to_pandas()

# Buat grafik
chart = alt.Chart(df_for_viz).mark_line(point=True).encode(
    x=alt.X('BULAN:N', 
            sort=list(bulan_order.keys()),
            axis=alt.Axis(
                labelAngle=0,  # Label horizontal
                labelLimit=150,  # Batas lebar label sebelum otomatis rotate
                labelOverlap="parity"  # Hindari tumpang tindih
            )),
    y=alt.Y('PUS:Q',
            scale=alt.Scale(domain=(y_min, y_max), nice=False),
            axis=alt.Axis(
                labelExpr='replace(format(datum.value, ",.0f"), ",", ".")',
                title='Jumlah PUS'
            )),
    tooltip=[
        alt.Tooltip('BULAN:N', title='Bulan'),
        alt.Tooltip('PUS_formatted:N', title='Jumlah PUS')
    ]
).properties(
    title='Tren Jumlah PUS di Sulawesi Barat', width = 600, height = 400
).configure_view(
    strokeWidth=0
).configure_axis(
    grid=False
).configure_point(
    size=100
).configure(
    padding=20,  # Tambah padding
    background='#f6f8fa'  # Warna background
)

chart

alt.Chart(...)

In [30]:

# Buat anotasi untuk titik awal
start_label = alt.Chart(df_for_viz).transform_filter(
    alt.datum.BULAN == df_for_viz['BULAN'].min()
).mark_text(
    align='center',
    dy=-20,          # Geser ke atas titik
    fontSize=14,
    fontWeight='bold',
    color='black',    # WARNA HITAM
).encode(
    x=alt.X('BULAN:N', sort=list(month_order.keys())),
    y=alt.Y('PUS:Q'),
    text=alt.Text('PUS_formatted:N')
)

# Buat anotasi untuk titik akhir
end_label = alt.Chart(df_for_viz).transform_filter(
    alt.datum.BULAN == df_for_viz['BULAN'].max()
).mark_text(
    align='center',
    dy=-20, 
    fontSize=14,
    fontWeight='bold',
    color='black',    # WARNA HITAM
).encode(
    x=alt.X('BULAN:N', sort=list(month_order.keys())),
    y=alt.Y('PUS:Q'),
    text=alt.Text('PUS_formatted:N')
)

# Gabungkan semua elemen
chart = alt.layer(
    # Base chart (garis dan point)
    alt.Chart(df_for_viz).mark_line(
        point=alt.OverlayMarkDef(
            size=200,
            filled=True,
            fillOpacity=1,
            color='#3498db'  # Warna point biru
        )
    ).encode(
        x=alt.X('BULAN:N', 
                sort=list(month_order.keys()),
                axis=alt.Axis(
                    labelAngle=0,
                    labelLimit=150,
                    labelOverlap="parity"
                )),
        y=alt.Y('PUS:Q',
                scale=alt.Scale(domain=(y_min, y_max), nice=False),
                axis=alt.Axis(
                    labelExpr='replace(format(datum.value, ",.0f"), ",", ".")',
                    title='Jumlah PUS'
                )),
        tooltip=[
            alt.Tooltip('BULAN:N', title='Bulan'),
            alt.Tooltip('PUS_formatted:N', title='Jumlah PUS')
        ]
    ),
    
    # Anotasi
    start_label,
    end_label
).properties(
    title='Tren Jumlah PUS di Sulawesi Barat', width = 600, height = 400
).configure_view(
    strokeWidth=0
).configure_axis(
    grid=False
).configure(
    padding=20,
    background='#f6f8fa'
)

chart

alt.LayerChart(...)

In [34]:
import pandas as pd
import altair as alt

# Data dummy dengan bulan kapital
data = {
    'BULAN': ['JANUARI', 'FEBRUARI', 'MARET', 'APRIL', 'MEI', 'JUNI', 
              'JULI', 'AGUSTUS', 'SEPTEMBER', 'OKTOBER', 'NOVEMBER', 'DESEMBER'],
    'PUS': [1250, 1320, 1410, 1380, 1450, 1500, 
            1520, 1570, 1600, 1620, 1680, 1700]
}

df_for_viz = pd.DataFrame(data)
df_for_viz['PUS_formatted'] = df_for_viz['PUS'].apply(lambda x: f"{x:,}".replace(',', '.'))

# Definisi urutan bulan (kapital)
month_order = {
    'JANUARI': 1, 'FEBRUARI': 2, 'MARET': 3, 'APRIL': 4,
    'MEI': 5, 'JUNI': 6, 'JULI': 7, 'AGUSTUS': 8,
    'SEPTEMBER': 9, 'OKTOBER': 10, 'NOVEMBER': 11, 'DESEMBER': 12
}

# Hitung y_min dan y_max
y_min = df_for_viz['PUS'].min()
y_max = df_for_viz['PUS'].max()

# === Base Chart ===
base = alt.Chart(df_for_viz).mark_line(
    point=alt.OverlayMarkDef(
        size=200,
        filled=True,
        fillOpacity=1,
        color='#3498db'
    )
).encode(
    x=alt.X('BULAN:N', 
            sort=list(month_order.keys()),
            axis=alt.Axis(
                labelAngle=90,
                labelLimit=200,
                labelOverlap="parity"
            )),
    y=alt.Y('PUS:Q',
            scale=alt.Scale(domain=(y_min, y_max), nice=False),
            axis=alt.Axis(
                labelExpr='replace(format(datum.value, ",.0f"), ",", ".")',
                title='Jumlah PUS'
            )),
    tooltip=[
        alt.Tooltip('BULAN:N', title='Bulan'),
        alt.Tooltip('PUS_formatted:N', title='Jumlah PUS')
    ]
)

# === Anotasi Label ===
start_label = alt.Chart(df_for_viz).transform_filter(
    alt.datum.BULAN == df_for_viz['BULAN'].min()
).mark_text(
    align='center',
    dy=-25,
    fontSize=14,
    fontWeight='bold',
    color='black'
).encode(
    x=alt.X('BULAN:N', sort=list(month_order.keys())),
    y=alt.Y('PUS:Q'),
    text=alt.Text('PUS_formatted:N')
)

end_label = alt.Chart(df_for_viz).transform_filter(
    alt.datum.BULAN == df_for_viz['BULAN'].max()
).mark_text(
    align='center',
    dy=-25,
    fontSize=14,
    fontWeight='bold',
    color='black'
).encode(
    x=alt.X('BULAN:N', sort=list(month_order.keys())),
    y=alt.Y('PUS:Q'),
    text=alt.Text('PUS_formatted:N')
)

# === Gabungkan Semua Elemen ===
chart = alt.layer(
    base,
    start_label,
    end_label
).properties(
    title='Tren Jumlah PUS di Sulawesi Barat',
    width=600,
    height=350
).configure_view(
    strokeWidth=0
).configure_axis(
    grid=False,
    labelFontSize=12,
    titleFontSize=14
).configure(
    padding=20,
    background='#f6f8fa'
)

chart.show()

alt.LayerChart(...)

In [35]:
alt.datum.BULAN == df_for_viz['BULAN'].max()

(datum.BULAN === 'SEPTEMBER')

In [ ]:
import polars as pl
import altair as alt

# Data dummy dengan Polars
data = {
    'BULAN': ['JANUARI', 'FEBRUARI', 'MARET', 'APRIL', 'MEI', 'JUNI', 
              'JULI', 'AGUSTUS', 'SEPTEMBER', 'OKTOBER', 'NOVEMBER', 'DESEMBER'],
    'PUS': [1250, 1320, 1410, 1380, 1450, 1500, 
            1520, 1570, 1600, 1620, 1680, 1700]
}

aggregated = pl.DataFrame(data)

# Definisi urutan bulan
month_order = {
    "JANUARI": 1, "FEBRUARI": 2, "MARET": 3, "APRIL": 4,
    "MEI": 5, "JUNI": 6, "JULI": 7, "AGUSTUS": 8,
    "SEPTEMBER": 9, "OKTOBER": 10, "NOVEMBER": 11, "DESEMBER": 12
}

# Proses data dengan Polars (PERBAIKAN DI SINI)
df_processed = (
    aggregated
    .with_columns(
        # Konversi bulan ke numerik
        pl.col("BULAN")
        .replace(month_order)
        .cast(pl.Int32)
        .alias("month_order")
    )
    .with_columns(
        # Perbaikan: Tambahkan return_dtype
        pl.col("PUS")
        .map_elements(
            lambda x: f"{x:,}".replace(",", "."),
            return_dtype=pl.Utf8  # TENTUKAN TIPE DATA RETURN
        )
        .alias("PUS_formatted")
    )
    .sort("month_order")
    .drop("month_order")
)

# Ambil bulan pertama dan terakhir
first_month = df_processed["BULAN"][0]
last_month = df_processed["BULAN"][-1]

# Hitung batas y-axis
y_min = df_processed["PUS"].min() * 0.95
y_max = df_processed["PUS"].max() * 1.05

# === Base Chart ===
base = alt.Chart(df_processed).mark_line(
    point=alt.OverlayMarkDef(size=200, filled=True, fillOpacity=1, color='#3498db')
).encode(
    x=alt.X('BULAN:N', 
            sort=list(month_order.keys()),
            axis=alt.Axis(labelAngle=0, labelLimit=200, labelOverlap="parity")),
    y=alt.Y('PUS:Q',
            scale=alt.Scale(domain=(y_min, y_max)),
            axis=alt.Axis(labelExpr='replace(format(datum.value, ",.0f"), ",", ".")', title='Jumlah PUS')),
    tooltip=[alt.Tooltip('BULAN:N'), alt.Tooltip('PUS_formatted:N')]
)

# === Anotasi Label ===
start_label = alt.Chart(df_processed).transform_filter(
    alt.datum.BULAN == first_month
).mark_text(
    align='center', dy=-25, fontSize=14, fontWeight='bold', color='blue'
).encode(
    x=alt.X('BULAN:N', sort=list(month_order.keys())),
    y=alt.Y('PUS:Q'),
    text=alt.Text('PUS_formatted:N')
)

end_label = alt.Chart(df_processed).transform_filter(
    alt.datum.BULAN == last_month
).mark_text(
    align='center', dy=-25, fontSize=14, fontWeight='bold', color='black'
).encode(
    x=alt.X('BULAN:N', sort=list(month_order.keys())),
    y=alt.Y('PUS:Q'),
    text=alt.Text('PUS_formatted:N')
)

# === Gabungkan Semua Elemen ===
chart = alt.layer(base, start_label, end_label).properties(
    title='Tren Jumlah PUS di Sulawesi Barat',
    width=600,
    height=350
).configure_view(
    strokeWidth=0
).configure_axis(
    grid=False,
    labelFontSize=12,
    titleFontSize=14
).configure(
    padding=20,
    background='#f6f8fa'
)

chart.show()

alt.LayerChart(...)

In [49]:
df_processed["BULAN"][-1]

'DESEMBER'